In [1]:
!pip install torchmetrics[multimodal]
!pip install transformers>=4.0
!pip install openai-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.7 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368607 sha256=25c84d157a567b6eb9c5c457e8316daeb4a0bf1d00eb697549bd3cb42a62917b
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip


In [2]:
import os
from google.colab import drive
import torch


In [3]:
#Connect to Google Drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [4]:
#General descriptors
watercourses = ['ditch','canal','brook','creek','river','stream']

#moods = ['tranquil','peaceful','beautiful','moody','raging']

loc_country = ['Canada','USA','Germany','Scotland','Iceland','Russia',
               'China','India','Brazil','Peru','Argentina','Nigeria','Zimbabwe',
               'South Africa','Australia','New Zealand']

loc_state = ['Alberta','British Colubmia','Ontario','Quebec','Alaska','New York',
             'Florida','Colorado','Montana','Iowa','Kentucky','Washington','Louisiana','California',
             'Maine','North Carolina','Georgia','Texas','Vermont']



loc_rivers = ['Colorado','Hudson','Columbia','Mississippi','Danube','Rhine','Seine','Thames','Amazon','Rio Negro',
              'Paraguay','Magdalena','Nile','Niger','Zambezi','Congo','Yangtze','Yamuna','Mekong','Ganges',
              'Sepik','Murray','Waikato','Darling']

#Morphological descriptors
morph_land = ['rural','urban','city','desert','field','grass','swamp','forest','mountain','canyon','valley']
morph_grad = ['tidal','flat','calm','pool','riffle','rapids','steep','waterfall']
morph_dim = ['narrow','wide','small','large']
morph_geol = ['clay','mud','sand','pebble','gravel','cobble','boulders','rock','shale','bedrock','limestone','granite']

#Environmental descriptors
env_water = ['clear','brown','blue','dirty','muddy','murky','tannic','whitewater']
env_flow = ['dry','drought','low flow','flowing','high flow','flooding']
env_weather = ['sunny','overcast','cloudy','rainy','snowy']
env_season = ['spring','summer','fall','autumn','winter']


In [9]:
import torch
import clip
from PIL import Image
import requests
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def calculate_clip_score(model, preprocess, image_path1, image_path2):

    # Load the images
    image1 = preprocess(Image.open(image_path1)).unsqueeze(0).to(device)
    image2 = preprocess(Image.open(image_path2)).unsqueeze(0).to(device)

    # Encode the images using the CLIP model
    with torch.no_grad():
        image_features1 = model.encode_image(image1)
        image_features2 = model.encode_image(image2)

    # Calculate the CLIP score (cosine similarity between the image embeddings)
    cosine_similarity = torch.nn.functional.cosine_similarity(image_features1, image_features2, dim=1)
    clip_score = cosine_similarity.item()

    return clip_score

In [14]:
def get_clip_score(model, preprocess, results_folder, prompt, num_imgs=30):

  scores = []

  for i in range(0,num_imgs):

    image_loc = "{}/{}/{}.jpg".format(results_folder,prompt,i)
    base_img = "{}/river/{}.jpg".format(results_folder,i)
    score = calculate_clip_score(model, preprocess, base_img, image_loc)
    scores.append(score)
    #print(f"CLIP score between the images: {score}")

  print('{:0.2f}+/-{:0.2f}'.format(np.mean(score),np.std(scores)))
  return scores


In [16]:
results_folder = '/content/drive/MyDrive/ControlNet-Experiment/Results2'
num_imgs = 30

'''
for watercourse in watercourses:

  #Generate prompt
  prompt = "{}".format(watercourse)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for country in loc_country:

  #Generate prompt
  prompt = "river {}".format(country)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

'''

for state in loc_state:

  #Generate prompt
  prompt = "river {}".format(state)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for river in loc_rivers:

  #Generate prompt
  prompt = "{} river".format(river)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for land in morph_land:

  #Generate prompt
  prompt = "{} river".format(land)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for grad in morph_grad:

  #Generate prompt
  prompt = "{} river".format(grad)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for dim in morph_dim:

  #Generate prompt
  prompt = "{} river".format(dim)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for geol in morph_geol:

  #Generate prompt
  prompt = "{} river".format(geol)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for water in env_water:

  #Generate prompt
  prompt = "{} river".format(water)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for flow in env_flow:

  #Generate prompt
  prompt = "{} river".format(flow)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for weather in env_weather:

  #Generate prompt
  prompt = "{} river".format(weather)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)

for season in env_season:

  #Generate prompt
  prompt = "{} river".format(season)
  print(prompt)
  c_score = get_clip_score(model,preprocess,results_folder,prompt)



river Alberta
0.91+/-0.08
river British Colubmia
0.88+/-0.09
river Ontario
0.83+/-0.10
river Quebec
0.85+/-0.09
river Alaska
0.82+/-0.08
river New York
0.67+/-0.07
river Florida
0.81+/-0.12
river Colorado
0.91+/-0.09
river Montana
0.88+/-0.10
river Iowa
0.75+/-0.10
river Kentucky
0.87+/-0.13
river Washington
0.89+/-0.08
river Louisiana
0.81+/-0.11
river California
0.81+/-0.12
river Maine
0.94+/-0.11
river North Carolina
0.74+/-0.13
river Georgia
0.88+/-0.10
river Texas
0.82+/-0.11
river Vermont
0.92+/-0.09
Colorado river
0.81+/-0.07
Hudson river
0.76+/-0.08
Columbia river
0.81+/-0.08
Mississippi river
0.76+/-0.08
Danube river
0.78+/-0.08
Rhine river
0.71+/-0.08
Seine river
0.66+/-0.07
Thames river
0.78+/-0.10
Amazon river
0.78+/-0.07
Rio Negro river
0.76+/-0.09
Paraguay river
0.76+/-0.08
Magdalena river
0.76+/-0.08
Nile river
0.81+/-0.09
Niger river
0.72+/-0.08
Zambezi river
0.80+/-0.06
Congo river
0.82+/-0.08
Yangtze river
0.66+/-0.09
Yamuna river
0.58+/-0.07
Mekong river
0.72+/-0.07


In [6]:
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

img = '/content/drive/MyDrive/ControlNet-Experiment/Results2/river/0.jpg'

image = preprocess(Image.open(img)).unsqueeze(0).to(device)

with torch.no_grad():
    image_features1 = model.encode_image(image)
